# Import des modules

In [1]:
import glob
import pandas as pd

# Lecture des données 

Données venant du Sirsé et cartosanté

In [12]:
liste = glob.glob('*.csv')
for i in range(len(liste)):
    if i == 0 :
        db = pd.read_csv(liste[i], skiprows=2, sep=';', encoding="utf-8", low_memory=False)
    else:
        db_temp = pd.read_csv(liste[i], skiprows=2, sep=';', encoding="utf-8", low_memory=False)
        try:
            db_temp.drop('Libellé',axis=1, inplace=True)
        except:
            pass
        db = db.merge(db_temp, left_on='Code', right_on='Code', how='inner')

Données de l'insee sur les codes de départements et regions

In [3]:
dep = pd.read_csv("../dep/communes-departement-region.csv", sep=",", encoding="utf-8")
#dep = dep.groupby(['code_commune_INSEE']).count().reset_index()
dep['code_commune_INSEE'] = dep['code_commune_INSEE'].apply(lambda x: '0'+str(x) if(len(str(x)) == 4) else str(x))
dep['code_postal'] = dep['code_postal'].apply(lambda x: '0'+str(x) if(len(str(x)) == 4) else str(x))
dep['code_dep'] = dep['code_commune_INSEE'].str[:2]
dep = dep[['code_dep','code_commune_INSEE', 
     'nom_departement','nom_region','code_postal',
     'code_region','longitude','latitude']]

# Modification des noms des colonnes

In [4]:
db.columns = ['Code','Nom_com', 'NG19_60>', 'NG19T',
              'Pop16_<18', 'NC16_15_64','NC16_15_24',
              'NRP16','RFM16','Pop16_>65','TN16','PM16']

# Jointure des données

In [7]:
merged_data = pd.merge(db,dep, left_on= "Code", right_on ="code_commune_INSEE", how="left")
data = pd.DataFrame(merged_data['Code'].value_counts() == 1)
liste_code_commune = data[data['Code']==True].index
merged_data_one = merged_data[merged_data['Code'].isin(liste_code_commune)]
#merged_data_one['code_region'] = merged_data_one['code_region'].astype(int).astype(str)
merged_data_one.drop(["Code"], axis=1, inplace=True)

C:\Users\joaka\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


# Vue sur les données

In [9]:
merged_data_one

,Nom_com,NG19_60>,NG19T,Pop16_<18,NC16_15_64,NC16_15_24,NRP16,RFM16,Pop16_>65,TN16,PM16,code_dep,code_commune_INSEE,nom_departement,nom_region,code_postal,code_region,longitude,latitude
0,L'Abergement-Clémenciat,0,0,194.0,33.0,8.0,8.0,23440.0,143.0,9.1,767.0,01,01001,Ain,Auvergne-Rhône-Alpes,01400,84.0,4.926114,46.153426
1,L'Abergement-de-Varey,0,0,62.0,10.0,4.0,0.0,24214.0,44.0,24.7,243.0,01,01002,Ain,Auvergne-Rhône-Alpes,01640,84.0,5.428017,46.009188
2,Ambérieu-en-Bugey,4,21,3369.0,1080.0,258.0,1305.0,19450.0,2322.0,15.6,14081.0,01,01004,Ain,Auvergne-Rhône-Alpes,01500,84.0,5.372926,45.960848
3,Ambérieux-en-Dombes,1,2,403.0,68.0,25.0,31.0,23280.0,257.0,8.4,1671.0,01,01005,Ain,Auvergne-Rhône-Alpes,01330,84.0,4.912273,45.996180
4,Ambléon,0,0,15.0,8.0,3.0,0.0,NaN,26.0,18.2,110.0,01,01006,Ain,Auvergne-Rhône-Alpes,01300,84.0,5.594320,45.749499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37957,Mtsamboro,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97,97612,Mayotte,Mayotte,97630,6.0,45.073680,-12.687507
37958,M'Tsangamouji,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97,97613,Mayotte,Mayotte,97650,6.0,45.087170,-12.751310
37961,Pamandzi,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97,97615,Mayotte,Mayotte,97615,6.0,45.284206,-12.796135
37962,Sada,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97,97616,Mayotte,Mayotte,97640,6.0,45.118550,-12.861165
